In [1]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using Plots
using BilevelTrajOpt

In [2]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
μ = 0.5
motion_type = :xyz
point = Point3D(default_frame(findbody(mechanism, "floor")), [0.,0.,0.])
normal = FreeVector3D(default_frame(findbody(mechanism, "floor")), [0.,0.,1.])
hs = HalfSpace(point, normal)
floor = Obstacle(hs, μ, motion_type)
obstacles = [floor]
env = parse_contacts(mechanism, urdf, obstacles)
x0 = MechanismState(mechanism)
Δt = 0.005
sim_data = get_sim_data(x0,env,Δt,true);

In [22]:
q0 = [1., 0., 0., 0., 0., 0., 0.1]
v0 = [0., 0., 0., 0., 0., 0.]

u0 = zeros(sim_data.num_v)
set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)
ctrl! = (u,t,x) -> u[:] = 0.
traj = BilevelTrajOpt.simulate(x0,env,sim_data.Δt,1,ctrl!,implicit_contact=false)
qnext = traj[1:sim_data.num_q,2]
vnext = traj[sim_data.num_q+1:sim_data.num_q+sim_data.num_v,2]
x_sol_exp = traj[sim_data.num_q+sim_data.num_v+2:end,2]

# qnext = q0
# vnext = v0

τ_ip, x_sol_ip, λ_ip, μ_ip, fx_ip = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=true);

τ_auglag, x_sol_auglag, λ_auglag, μ_auglag, fx_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);

# display("Explicit")
# display(x_sol_exp)
display("IP")
display(x_sol_ip)
display("Aug Lag")
display(x_sol_auglag)

"IP"

6-element Array{Float64,1}:
 1.16769e-8
 1.16769e-8
 1.16769e-8
 1.16769e-8
 0.00228923
 7.38252e-8

"Aug Lag"

6-element Array{Float64,1}:
 0.00693148
 0.00693148
 0.00693148
 0.00693148
 0.00693147
 0.00693151

Solve_Succeeded
Solve_Succeeded


In [5]:
display("IP")
display(x_sol_ip)
display("Aug Lag")
display(x_sol_auglag)

"IP"

6-element Array{Float64,1}:
 0.0166248
 0.0166248
 0.0166248
 0.0166248
 0.0411736
 1.0      

"Aug Lag"

6-element Array{Float64,1}:
 0.125     
 0.125     
 0.125     
 0.125     
 0.00693147
 1.0       

In [7]:
# autodiff 
J_auto = ForwardDiff.jacobian(qv -> solve_implicit_contact_τ(sim_data,q0,v0,u0,qv[1:sim_data.num_q],qv[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[2], vcat(qnext,vnext))

# numerical
ϵ = 1e-12
J_num = zeros(size(J_auto))
qv = vcat(qnext,vnext)
for i = 1:length(qv)
    δ = zeros(length(qv))
    δ[i] = ϵ
    J_num[:,i] = (solve_implicit_contact_τ(sim_data,q0,v0,u0,(qv+δ)[1:sim_data.num_q],(qv+δ)[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[2] - x_sol_auglag)/ϵ
end

display(J_auto)
display(J_num)

6×13 Array{Float64,2}:
 0.0   9.73813e-26   3.37201e-17   7.83223e-25  …   -1.59078e-7   2.54841    
 0.0  -8.43907e-18   3.07818e-24   1.02876e-24      10.1936       2.54841    
 0.0   9.73813e-26  -3.37201e-17  -7.83223e-25      -1.59078e-7   2.54841    
 0.0   8.43907e-18   3.07818e-24  -1.02876e-24     -10.1936       2.54841    
 0.0   4.14683e-35  -1.51847e-36  -2.99409e-51      -4.7738e-6    6.08655e-29
 0.0   7.79053e-25   2.46255e-23  -2.35599e-50  …   -1.90894e-6  20.3874     

6×13 Array{Float64,2}:
 0.0  -5.55112e-5  -5.55112e-5  -5.55112e-5  …    0.0          2.54838   
 0.0  -2.77556e-5  -2.77556e-5  -2.77556e-5      10.1936       2.54838   
 0.0   0.0          0.0          0.0              2.77556e-5   2.54841   
 0.0  -5.55112e-5   0.0          0.0            -10.1937       2.54838   
 0.0   0.0          0.0          1.73472e-6       6.07153e-6   1.73472e-6
 0.0   0.0          0.0          0.0         …    0.0         20.3872    

In [10]:
maximum(abs.(J_auto-J_num))

0.00011239110444094536